In [70]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import pymongo
from dotenv import load_dotenv

In [71]:
mongo_uri = os.getenv("MONGO_URI")
mongo_client = pymongo.MongoClient(mongo_uri)

db_cow_monitoring = mongo_client["cow_monitoring"]
collection_cow_data = db_cow_monitoring["CowData"]

In [72]:
cow_data = pd.DataFrame(list(collection_cow_data.find()))

In [73]:
cow_data_dict = {cow_id: cow_data[cow_data['slave_id'] == cow_id] for cow_id in cow_data['slave_id'].unique()}

In [74]:
# Function to create records
def create_record_from_data(cow_data):
    grouped = cow_data.groupby('timestamp')  # Group by timestamp
    records = []

    # For each group of records with the same timestamp
    for timestamp, group in grouped:
        record = {'timestamp': timestamp}
        
        for _, row in group.iterrows():
            record[row['parameter']] = row['value']
        
        records.append(record)
    
    return pd.DataFrame(records).ffill()

In [75]:
columns = ['timestamp', 'slave_id', 'body_temperature', 'activity_level', 'milk_production', 'body_condition_score', 'estrous_cycle', 'feed_intake', 'rumen_ph']
df = pd.DataFrame(columns=columns)
for i in cow_data_dict:
    new_record = create_record_from_data(cow_data_dict[i])
    new_record['slave_id'] = i
    df = pd.concat([df, new_record], ignore_index=True)

/var/folders/hm/g79r9phd7ld_128zswnn0qhh0000gn/T/ipykernel_63344/2729918074.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_record], ignore_index=True)
